In [1]:
import random
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import trange
from torch.autograd import Variable

In [2]:
import gensim.downloader
word2vec_google_news: gensim.models.keyedvectors.KeyedVectors = gensim.downloader.load('word2vec-google-news-300')

In [3]:
word2vec_google_news.add_vector("<pad>", np.zeros(300))
padding_idx = word2vec_google_news.key_to_index["<pad>"]
embedding_weights = torch.FloatTensor(word2vec_google_news.vectors)
vocab = word2vec_google_news.key_to_index

In [4]:
# print(f"padding_idx: {padding_idx}")
# print(len(embedding_weights))
# print(embedding_weights)

In [5]:
# tags_path = "stanford_cs230_medium_data/tags.txt"
tags_path = "stanford_cs230_small_data/tags.txt"
tag_map = {}
with open(tags_path, encoding="utf-8") as f:
    for i, l in enumerate(f.read().splitlines()):
        tag_map[l] = i
tag_map

{'O': 0, 'I-geo': 1, 'I-gpe': 2, 'I-per': 3, 'I-org': 4, 'I-tim': 5}

In [6]:
# train_sentences_file = "stanford_cs230_medium_data/train/sentences.txt"
# train_labels_file = "stanford_cs230_medium_data/train/labels.txt"
train_sentences_file = "stanford_cs230_small_data/test/sentences.txt"
train_labels_file = "stanford_cs230_small_data/test/labels.txt"

train_sentences = []
train_labels = []

with open(train_sentences_file, encoding="utf-8") as f:
    for sentence in f.read().splitlines():
        #replace each token by its index if it is in vocab
        #else use index of UNK
        s = [vocab[token] if token in vocab
             else vocab['UNK']
             for token in sentence.split(' ')]
        train_sentences.append(s)

with open(train_labels_file) as f:
    for sentence in f.read().splitlines():
        #replace each label by its index
        l = [tag_map[label] for label in sentence.split(' ')]
        train_labels.append(l)
        
print(f"train_sentences: {train_sentences}")
print(f"train_labels: {train_labels}")

train_sentences: [[332, 11, 708, 98307, 6318, 3211, 1, 3067, 98307, 1759, 15201, 2031, 866354, 22076, 9, 22, 4, 12597, 18, 11, 791, 1020, 98307], [57, 1229, 2728, 3262, 336, 13, 16, 101276, 98307], [69307, 979, 1020, 5, 11, 3499, 11357, 273, 1, 98307, 496, 98307, 68, 98307, 1238, 1009, 98307], [73, 1275, 1080, 229, 24, 266, 5, 2125, 618, 98307, 366, 869, 5, 12054, 98307, 29255, 4483, 4013, 98307, 677, 1275, 1080, 8622, 98307], [7, 17027, 14046, 2599, 2, 12054, 9, 247, 17, 11, 954, 98307, 7106, 2125, 8767, 349, 28, 116, 130, 1227, 65, 1353, 1, 2821, 8, 12054, 98307], [51, 9, 618, 130, 1227, 2353, 98307, 522, 2196, 5, 12054, 18, 11, 1217, 8767, 2799, 792, 98307], [7, 229, 53, 2128, 14046, 618, 98307, 6446, 11, 131, 371, 601, 1215, 18, 12054, 98307, 467, 3181, 98307], [7, 1080, 229, 11092, 12054, 98307, 121, 98307, 3327, 1, 2039, 914, 10708, 98307, 1275, 1080, 8622, 98307, 755, 103, 78, 5336, 756, 1, 12054, 98307], [1290, 247, 159, 7144, 21, 496, 98307, 536, 98307, 11, 9330, 2434, 363, 18

In [7]:
# val_sentences_file = "stanford_cs230_medium_data/val/sentences.txt"
# val_labels_file = "stanford_cs230_medium_data/val/labels.txt"
val_sentences_file = "stanford_cs230_small_data/test/sentences.txt"
val_labels_file = "stanford_cs230_small_data/test/labels.txt"

val_sentences = []
val_labels = []

with open(val_sentences_file, encoding="utf-8") as f:
    for sentence in f.read().splitlines():
        #replace each token by its index if it is in vocab
        #else use index of UNK
        s = [vocab[token] if token in vocab
             else vocab['UNK']
             for token in sentence.split(' ')]
        val_sentences.append(s)

with open(val_labels_file, encoding="utf-8") as f:
    for sentence in f.read().splitlines():
        #replace each label by its index
        l = [tag_map[label] for label in sentence.split(' ')]
        val_labels.append(l)

print(f"val_sentences: {val_sentences}")
print(f"val_labels: {val_labels}")

val_sentences: [[332, 11, 708, 98307, 6318, 3211, 1, 3067, 98307, 1759, 15201, 2031, 866354, 22076, 9, 22, 4, 12597, 18, 11, 791, 1020, 98307], [57, 1229, 2728, 3262, 336, 13, 16, 101276, 98307], [69307, 979, 1020, 5, 11, 3499, 11357, 273, 1, 98307, 496, 98307, 68, 98307, 1238, 1009, 98307], [73, 1275, 1080, 229, 24, 266, 5, 2125, 618, 98307, 366, 869, 5, 12054, 98307, 29255, 4483, 4013, 98307, 677, 1275, 1080, 8622, 98307], [7, 17027, 14046, 2599, 2, 12054, 9, 247, 17, 11, 954, 98307, 7106, 2125, 8767, 349, 28, 116, 130, 1227, 65, 1353, 1, 2821, 8, 12054, 98307], [51, 9, 618, 130, 1227, 2353, 98307, 522, 2196, 5, 12054, 18, 11, 1217, 8767, 2799, 792, 98307], [7, 229, 53, 2128, 14046, 618, 98307, 6446, 11, 131, 371, 601, 1215, 18, 12054, 98307, 467, 3181, 98307], [7, 1080, 229, 11092, 12054, 98307, 121, 98307, 3327, 1, 2039, 914, 10708, 98307, 1275, 1080, 8622, 98307, 755, 103, 78, 5336, 756, 1, 12054, 98307], [1290, 247, 159, 7144, 21, 496, 98307, 536, 98307, 11, 9330, 2434, 363, 18, 

In [8]:
def data_iterator(sentences, labels, total_size: int, batch_size: int, shuffle: bool=False):
    # make a list that decides the order in which we go over the data- this avoids explicit shuffling of data
    order = list(range(total_size))
    if shuffle:
        random.seed(230)
        random.shuffle(order)

    # one pass over data
    for i in range((total_size+1)//batch_size):
        # fetch sentences and tags
        batch_sentences = [sentences[idx] for idx in order[i*batch_size:(i+1)*batch_size]]
        batch_tags = [labels[idx] for idx in order[i*batch_size:(i+1)*batch_size]]

        # compute length of longest sentence in batch
        batch_max_len = max([len(s) for s in batch_sentences])

        # prepare a numpy array with the data, initialising the data with pad_ind and all labels with -1
        # initialising labels to -1 differentiates tokens with tags from PADding tokens
        batch_data = vocab['<pad>']*np.ones((len(batch_sentences), batch_max_len))
        batch_labels = -1*np.ones((len(batch_sentences), batch_max_len))

        # copy the data to the numpy array
        for j in range(len(batch_sentences)):
            cur_len = len(batch_sentences[j])
            batch_data[j][:cur_len] = batch_sentences[j]
            batch_labels[j][:cur_len] = batch_tags[j]

        # since all data are indices, we convert them to torch LongTensors
        batch_data, batch_labels = torch.LongTensor(batch_data), torch.LongTensor(batch_labels)

        # shift tensors to GPU if available
        # if params.cuda:
        #     batch_data, batch_labels = batch_data.cuda(), batch_labels.cuda()

        # convert them to Variables to record operations in the computational graph
        batch_data, batch_labels = Variable(batch_data), Variable(batch_labels)

        yield batch_data, batch_labels, batch_sentences

In [9]:
class Net(nn.Module):
    """
    This is the standard way to define your own network in PyTorch. You typically choose the components
    (e.g. LSTMs, linear layers etc.) of your network in the __init__ function. You then apply these layers
    on the input step-by-step in the forward function. You can use torch.nn.functional to apply functions
    such as F.relu, F.sigmoid, F.softmax. Be careful to ensure your dimensions are correct after each step.

    You are encouraged to have a look at the network in pytorch/vision/model/net.py to get a better sense of how
    you can go about defining your own network.

    The documentation for all the various components available to you is here: http://pytorch.org/docs/master/nn.html
    """

    def __init__(self, embedding_weights, embedding_dim, lstm_hidden_dim, number_of_tags):
        """
        We define an recurrent network that predicts the NER tags for each token in the sentence. The components
        required are:

        - an embedding layer: this layer maps each index in range(params.vocab_size) to a params.embedding_dim vector
        - lstm: applying the LSTM on the sequential input returns an output for each token in the sentence
        - fc: a fully connected layer that converts the LSTM output for each token to a distribution over NER tags

        Args:
            params: (Params) contains vocab_size, embedding_dim, lstm_hidden_dim
        """
        super(Net, self).__init__()

        # the embedding takes as input the vocab_size and the embedding_dim
        self.embedding = nn.Embedding.from_pretrained(embedding_weights, padding_idx=3_000_000)
        # self.embedding = nn.Embedding.from_pretrained(embedding_weights)

        # the LSTM takes as input the size of its input (embedding_dim), its hidden size
        # for more details on how to use it, check out the documentation
        self.lstm = nn.LSTM(embedding_dim,
                            lstm_hidden_dim, batch_first=True)

        # the fully connected layer transforms the output to give the final output layer
        self.fc = nn.Linear(lstm_hidden_dim, number_of_tags)

    def forward(self, s):
        """
        This function defines how we use the components of our network to operate on an input batch.

        Args:
            s: (Variable) contains a batch of sentences, of dimension batch_size x seq_len, where seq_len is
               the length of the longest sentence in the batch. For sentences shorter than seq_len, the remaining
               tokens are PADding tokens. Each row is a sentence with each element corresponding to the index of
               the token in the vocab.

        Returns:
            out: (Variable) dimension batch_size*seq_len x num_tags with the log probabilities of tokens for each token
                 of each sentence.

        Note: the dimensions after each step are provided
        """
        #                                -> batch_size x seq_len
        # apply the embedding layer that maps each token to its embedding
        # dim: batch_size x seq_len x embedding_dim
        s = self.embedding(s)

        # run the LSTM along the sentences of length seq_len
        # dim: batch_size x seq_len x lstm_hidden_dim
        s, _ = self.lstm(s)

        # make the Variable contiguous in memory (a PyTorch artefact)
        s = s.contiguous()

        # reshape the Variable so that each row contains one token
        # dim: batch_size*seq_len x lstm_hidden_dim
        s = s.view(-1, s.shape[2])

        # apply the fully connected layer and obtain the output (before softmax) for each token
        s = self.fc(s)                   # dim: batch_size*seq_len x num_tags

        # apply log softmax on each token's output (this is recommended over applying softmax
        # since it is numerically more stable)
        return F.log_softmax(s, dim=1)   # dim: batch_size*seq_len x num_tags

In [10]:
def loss_fn(outputs, labels):
    """
    Compute the cross entropy loss given outputs from the model and labels for all tokens. Exclude loss terms
    for PADding tokens.

    Args:
        outputs: (Variable) dimension batch_size*seq_len x num_tags - log softmax output of the model
        labels: (Variable) dimension batch_size x seq_len where each element is either a label in [0, 1, ... num_tag-1],
                or -1 in case it is a PADding token.

    Returns:
        loss: (Variable) cross entropy loss for all tokens in the batch

    Note: you may use a standard loss function from http://pytorch.org/docs/master/nn.html#loss-functions. This example
          demonstrates how you can easily define a custom loss function.
    """

    # reshape labels to give a flat vector of length batch_size*seq_len
    labels = labels.view(-1)

    # since PADding tokens have label -1, we can generate a mask to exclude the loss from those terms
    mask = (labels >= 0).float()

    # indexing with negative values is not supported. Since PADded tokens have label -1, we convert them to a positive
    # number. This does not affect training, since we ignore the PADded tokens with the mask.
    labels = labels % outputs.shape[1]

    num_tokens = int(torch.sum(mask))

    # compute cross entropy loss for all tokens (except PADding tokens), by multiplying with mask.
    return -torch.sum(outputs[range(outputs.shape[0]), labels]*mask)/num_tokens

In [11]:
class RunningAverage:
    """A simple class that maintains the running average of a quantity

    Example:
    ```
    loss_avg = RunningAverage()
    loss_avg.update(2)
    loss_avg.update(4)
    loss_avg() = 3
    ```
    """

    def __init__(self):
        self.steps = 0
        self.total = 0

    def update(self, val):
        self.total += val
        self.steps += 1

    def __call__(self):
        return self.total / float(self.steps)

In [12]:
def train(model, optimizer, loss_fn, data_iterator, metrics, num_steps):
    """Train the model on `num_steps` batches

    Args:
        model: (torch.nn.Module) the neural network
        optimizer: (torch.optim) optimizer for parameters of model
        loss_fn: a function that takes batch_output and batch_labels and computes the loss for the batch
        data_iterator: (generator) a generator that generates batches of data and labels
        metrics: (dict) a dictionary of functions that compute a metric using the output and labels of each batch
        params: (Params) hyperparameters
        num_steps: (int) number of batches to train on, each of size params.batch_size
    """

    # set model to training mode
    model.train()

    # summary for current training loop and a running average object for loss
    summ = []
    loss_avg = RunningAverage()

    # Use tqdm for progress bar
    t = trange(num_steps)
    for i in t:
        # fetch the next training batch
        train_batch, labels_batch, _ = next(data_iterator)

        # compute model output and loss
        output_batch = model(train_batch)
        loss = loss_fn(output_batch, labels_batch)

        # clear previous gradients, compute gradients of all variables wrt loss
        optimizer.zero_grad()
        loss.backward()

        # performs updates using calculated gradients
        optimizer.step()

        # Evaluate summaries only once in a while
        if i % 10 == 0:
            # extract data from torch Variable, move to cpu, convert to numpy arrays
            output_batch = output_batch.data.cpu().numpy()
            labels_batch = labels_batch.data.cpu().numpy()

            # compute all metrics on this batch
            summary_batch = {metric: metrics[metric](output_batch, labels_batch)
                             for metric in metrics}
            summary_batch['loss'] = loss.item()
            summ.append(summary_batch)

        # update the average loss
        loss_avg.update(loss.item())
        t.set_postfix(loss='{:05.3f}'.format(loss_avg()))

    # compute mean of all metrics in summary
    metrics_mean = {metric: np.mean([x[metric]
                                     for x in summ]) for metric in summ[0]}
    metrics_string = " ; ".join("{}: {:05.3f}".format(k, v)
                                for k, v in metrics_mean.items())
    print("- Train metrics: " + metrics_string)

In [13]:
def evaluate(model, loss_fn, data_iterator, metrics, num_steps):
    """Evaluate the model on `num_steps` batches.

    Args:
        model: (torch.nn.Module) the neural network
        loss_fn: a function that takes batch_output and batch_labels and computes the loss for the batch
        data_iterator: (generator) a generator that generates batches of data and labels
        metrics: (dict) a dictionary of functions that compute a metric using the output and labels of each batch
        params: (Params) hyperparameters
        num_steps: (int) number of batches to train on, each of size params.batch_size
    """

    # set model to evaluation mode
    model.eval()

    # summary for current eval loop
    summ = []

    # compute metrics over the dataset
    for _ in range(num_steps):
        # fetch the next evaluation batch
        data_batch, labels_batch, _ = next(data_iterator)

        # compute model output
        output_batch = model(data_batch)
        loss = loss_fn(output_batch, labels_batch)

        # extract data from torch Variable, move to cpu, convert to numpy arrays
        output_batch = output_batch.data.cpu().numpy()
        labels_batch = labels_batch.data.cpu().numpy()

        # compute all metrics on this batch
        summary_batch = {metric: metrics[metric](output_batch, labels_batch)
                         for metric in metrics}
        summary_batch['loss'] = loss.item()
        summ.append(summary_batch)

    # compute mean of all metrics in summary
    metrics_mean = {metric:np.mean([x[metric] for x in summ]) for metric in summ[0]}
    metrics_string = " ; ".join("{}: {:05.3f}".format(k, v) for k, v in metrics_mean.items())
    print("- Eval metrics : " + metrics_string)
    return metrics_mean

In [14]:
def train_and_evaluate(
        model,
        train_sentences,
        train_labels,
        val_sentences,
        val_labels,
        num_epochs: int,
        batch_size: int,
        optimizer,
        loss_fn,
        metrics
):
    for epoch in range(num_epochs):
        # Run one epoch
        print("Epoch {}/{}".format(epoch + 1, num_epochs))

        # compute number of batches in one epoch (one full pass over the training set)
        num_steps = (len(train_sentences) + 1) // batch_size
        train_data_iterator = data_iterator(
            train_sentences, train_labels, len(train_sentences), batch_size, shuffle=True)
        train(model, optimizer, loss_fn, train_data_iterator,
              metrics, num_steps)

        # Evaluate for one epoch on validation set
        num_steps = (len(val_sentences) + 1) // batch_size
        val_data_iterator = data_iterator(
            val_sentences, val_labels, len(val_sentences), batch_size, shuffle=False)
        val_metrics = evaluate(
            model, loss_fn, val_data_iterator, metrics, num_steps)

In [15]:
def accuracy(outputs, labels):
    """
    Compute the accuracy, given the outputs and labels for all tokens. Exclude PADding terms.

    Args:
        outputs: (np.ndarray) dimension batch_size*seq_len x num_tags - log softmax output of the model
        labels: (np.ndarray) dimension batch_size x seq_len where each element is either a label in
                [0, 1, ... num_tag-1], or -1 in case it is a PADding token.

    Returns: (float) accuracy in [0,1]
    """

    # reshape labels to give a flat vector of length batch_size*seq_len
    labels = labels.ravel()

    # since PADding tokens have label -1, we can generate a mask to exclude the loss from those terms
    mask = (labels >= 0)

    # np.argmax gives us the class predicted for each token by the model
    outputs = np.argmax(outputs, axis=1)

    # compare outputs with labels and divide by number of tokens (excluding PADding tokens)
    return np.sum(outputs == labels)/float(np.sum(mask))


metrics = {
    'accuracy': accuracy,
    # could add more metrics such as accuracy for each token type
}

In [16]:
word2vec_google_news["<pad>"] = np.zeros(300)
embedding_weights = torch.FloatTensor(word2vec_google_news.vectors)

In [17]:
# manually change vocab size (unique no. of words) and change label size (unique no. of labels) for now
model = Net(embedding_weights, 300, 300, 6)
optimizer = optim.Adam(model.parameters(), lr=0.001)
train_and_evaluate(model, train_sentences, train_labels, val_sentences, val_labels, 100, 5, optimizer, loss_fn, metrics)

if (os.path.isfile("model_weights.pth")):
    model.load_state_dict(torch.load('model_weights.pth'))
else:
    torch.save(model.state_dict(), 'model_weights.pth')


Epoch 1/100


100%|██████████| 2/2 [00:00<00:00, 66.67it/s, loss=1.754]


- Train metrics: accuracy: 0.106 ; loss: 1.781
- Eval metrics : accuracy: 0.797 ; loss: 1.631
Epoch 2/100


100%|██████████| 2/2 [00:00<00:00, 153.87it/s, loss=1.590]


- Train metrics: accuracy: 0.894 ; loss: 1.599
- Eval metrics : accuracy: 0.802 ; loss: 1.422
Epoch 3/100


...

- Train metrics: accuracy: 1.000 ; loss: 0.003
- Eval metrics : accuracy: 1.000 ; loss: 0.003
Epoch 99/100


100%|██████████| 2/2 [00:00<00:00, 153.85it/s, loss=0.003]


- Train metrics: accuracy: 1.000 ; loss: 0.003
- Eval metrics : accuracy: 1.000 ; loss: 0.003
Epoch 100/100


100%|██████████| 2/2 [00:00<00:00, 166.68it/s, loss=0.003]


- Train metrics: accuracy: 1.000 ; loss: 0.003
- Eval metrics : accuracy: 1.000 ; loss: 0.003


In [18]:
# test_sentences_file = "stanford_cs230_medium_data/test/sentences.txt"
# test_labels_file = "stanford_cs230_medium_data/test/labels.txt"
test_sentences_file = "stanford_cs230_small_data/test/sentences.txt"
test_labels_file = "stanford_cs230_small_data/test/labels.txt"

sentences_w_words = []
labels = []

test_sentences = []
test_labels = []

with open(test_sentences_file) as f:
    for sentence_words in f.read().splitlines():
        #replace each token by its index if it is in vocab
        #else use index of UNK
        sentences_w_words.append(sentence_words)
        s = [vocab[token] if token in vocab
             else vocab['UNK']
             for token in sentence_words.split(' ')]
        test_sentences.append(s)

with open(test_labels_file) as f:
    for sentence_labels in f.read().splitlines():
        #replace each label by its index
        l = [tag_map[label] for label in sentence_labels.split(' ')]
        test_labels.append(l)
        labels.append(sentence_labels)

print(f"sentences_w_words: {sentences_w_words}")
print(f"labels: {labels}")
print(f"test_sentences: {test_sentences}")
print(f"test_labels: {test_labels}")

sentences_w_words: ['At the Group of Eight summit in Scotland , Japanese Prime Minister Junichiro Koizumi said he is outraged by the London attacks .', 'He noted terrorist acts must not be forgivable .', 'Sarin gas attacks on the Tokyo subway system in 1995 killed 12 people and injured thousands .', 'A human rights group has called on Asian leaders to increase pressure on Burma to hasten democratic reforms and stop human rights abuses .', 'The Alternative ASEAN Network for Burma said officials from the Association of Southeast Asian Nations meeting this week should consider new options in dealing with Burma .', 'It said leaders should consider supporting a possible resolution on Burma by the United Nations Security Council .', "The group also urged ASEAN leaders to acknowledge the many security problems caused by Burma 's military regime .", "The rights group accuses Burma 's government of involvement in illegal drug trafficking and human rights abuses , especially against some ethnic 

In [19]:
print(len(test_sentences))
test_data_iterator = data_iterator(test_sentences, test_labels, len(test_sentences), 1, shuffle=False)
test_batch, labels_batch, abc = next(test_data_iterator)
model_output = model(test_batch)
print(model_output)
print(model_output.size())

10
tensor([[-1.9503e-02, -5.7721e+00, -6.5976e+00, -6.1075e+00, -4.5781e+00,
         -6.0591e+00],
        [-6.6907e-03, -6.9312e+00, -9.5465e+00, -8.9604e+00, -5.2664e+00,
         -8.0176e+00],
        [-5.1182e+00, -5.3215e+00, -9.3186e+00, -8.4371e+00, -1.2099e-02,
         -7.0733e+00],
        [-7.9006e+00, -8.3355e+00, -1.0205e+01, -9.6033e+00, -1.2172e-03,
         -7.5976e+00],
        [-7.1119e+00, -8.3138e+00, -9.9251e+00, -9.3340e+00, -1.3209e-03,
         -9.0097e+00],
        [-4.4683e+00, -6.4885e+00, -8.7820e+00, -8.1045e+00, -1.3622e-02,
         -9.3646e+00],
        [-9.1961e-03, -6.8682e+00, -9.8850e+00, -9.7032e+00, -4.8316e+00,
         -1.0488e+01],
        [-4.4696e+00, -2.3224e-02, -4.9887e+00, -8.6717e+00, -5.4459e+00,
         -8.4939e+00],
        [-5.7030e-03, -6.2460e+00, -6.0090e+00, -8.4167e+00, -7.4359e+00,
         -7.6393e+00],
        [-6.3135e+00, -4.8082e+00, -2.4245e-02, -4.4485e+00, -7.4930e+00,
         -6.3617e+00],
        [-5.6175e+00, -8.48

In [20]:
print(f"model_output \n {model_output}")
# predicted_labels = torch.argmax(model_output, dim=1)
# print(f"predicted_labels \n {predicted_labels}")
# predicted_labels = torch.argmax(torch.abs(model_output), dim=1)
# print(f"predicted_labels \n {predicted_labels}")
predicted_labels = np.argmax(model_output.detach().numpy(), axis=1)
print(f"sentences_w_words \n {abc}")
print(f"predicted_labels \n {predicted_labels}")
# print(f"correct_labels \n {}")

model_output 
 tensor([[-1.9503e-02, -5.7721e+00, -6.5976e+00, -6.1075e+00, -4.5781e+00,
         -6.0591e+00],
        [-6.6907e-03, -6.9312e+00, -9.5465e+00, -8.9604e+00, -5.2664e+00,
         -8.0176e+00],
        [-5.1182e+00, -5.3215e+00, -9.3186e+00, -8.4371e+00, -1.2099e-02,
         -7.0733e+00],
        [-7.9006e+00, -8.3355e+00, -1.0205e+01, -9.6033e+00, -1.2172e-03,
         -7.5976e+00],
        [-7.1119e+00, -8.3138e+00, -9.9251e+00, -9.3340e+00, -1.3209e-03,
         -9.0097e+00],
        [-4.4683e+00, -6.4885e+00, -8.7820e+00, -8.1045e+00, -1.3622e-02,
         -9.3646e+00],
        [-9.1961e-03, -6.8682e+00, -9.8850e+00, -9.7032e+00, -4.8316e+00,
         -1.0488e+01],
        [-4.4696e+00, -2.3224e-02, -4.9887e+00, -8.6717e+00, -5.4459e+00,
         -8.4939e+00],
        [-5.7030e-03, -6.2460e+00, -6.0090e+00, -8.4167e+00, -7.4359e+00,
         -7.6393e+00],
        [-6.3135e+00, -4.8082e+00, -2.4245e-02, -4.4485e+00, -7.4930e+00,
         -6.3617e+00],
        [-5.617